In [14]:
library(bigrquery)
library(dplyr)

projectid = "spot-pet-production"

# Set your query
#sql <- "SELECT * FROM `spot-pet-us.data_warehouse_us.data_partnerize_clicks`"

# Run the query and store the data in a tibble
#tb <- bq_project_query(projectid, sql)


#database_data <- bq_table_download(tb)
#database_data <- as.data.frame(database_data)
#nrow(database_data)

library(httr)
library(stringr)

todays_date <- as.character(Sys.Date())

todays_date <- as.Date(todays_date , "%Y-%m-%d")

tomorrow_date <- todays_date + 1

#yesterdays_date <- todays_date - 2

tomorrow_date <- as.character(tomorrow_date)

#yesterdays_date <- as.character(yesterdays_date)

URL <- "https://api.performancehorizon.com/reporting/export/export/click.csv?start_date=2020-06-01+00%3A00%3A00&end_date=2021-11-24+00%3A00%3A00&campaign_id=1011l3888&convert_currency=USD&ref_conversion_metric_id%5B%5D=2"
URL <- str_replace(URL, "(?<=end_date\\=)\\d{4}-\\d{2}-\\d{2}", tomorrow_date)
#URL <- str_replace(URL, "(?<=start_date\\=)\\d{4}-\\d{2}-\\d{2}", yesterdays_date)

data <- GET(URL, authenticate("8GkJjNy4wo","2L07IDNE"))
new_data <- read.csv(text=content(data, type = "text", encoding="UTF-8"))
job_id <- new_data$job_id
url <- "https://api.performancehorizon.com/job/x/response.csv"
url2 <- gsub("x", job_id, url)

######Wait 2 min to run this####
Sys.sleep(450)
data <- GET(url2, authenticate("8GkJjNy4wo","2L07IDNE"))
new_data <- read.csv(text=content(data, type = "text", encoding="UTF-8"))

#nrow(new_data)

new_data$set_time <- as.POSIXct(new_data$set_time, format = "%Y-%m-%d %H:%M:%S")
new_data$last_used <- as.POSIXct(new_data$set_time, format = "%Y-%m-%d %H:%M:%S")

#different.names <- (!new_data$click_id %in% database_data$click_id)
#data_to_append <- new_data[different.names,]

dataset_name <- 'flow_tables_exec_dash'
partnerize_table = bq_table(project = projectid, dataset = dataset_name, table = 'data_partnerize_clicks')

new_data$creative_id <- as.character(new_data$creative_id)
new_data$specific_creative_id <- as.character(new_data$creative_id)
new_data$ref_origin <- NULL

bq_table_upload(x=partnerize_table, values= new_data, create_disposition='CREATE_IF_NEEDED', write_disposition='WRITE_TRUNCATE')
sql2 <- "Update `spot-pet-production.flow_tables_exec_dash.data_refreshes_time` set time_refreshed=DATETIME(CURRENT_TIMESTAMP(), 'America/New_York') WHERE source = 'Partnerize_clicks'"
tb <- bq_project_query(projectid, sql2)

##### Run till here
print("update completed")

ERROR: Error in read.table(file = file, header = header, sep = sep, quote = quote, : no lines available in input
